In [1]:
import sys

In [2]:
sys.path.append('/Users/jwu/Documents/5400/data/cvxEDA-master/src') #modify abs path to your /src

In [3]:
#pip install cvxopt
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import scipy.signal as scisig
import scipy.stats
import cvxEDA

In [4]:
# E4 (wrist) Sampling Frequencies
fs_dict = {'ACC': 32, 'BVP': 64, 'EDA': 4, 'TEMP': 4, 'label': 700, 'Resp': 700}
WINDOW_IN_SECONDS = 30
label_dict = {'baseline': 1, 'stress': 2, 'amusement': 0}
int_to_label = {1: 'baseline', 2: 'stress', 0: 'amusement'}
feat_names = None
savePath = 'data'
subject_feature_path = '/subject_feats'

In [5]:
# check whether path exists
def ensure_directory_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)

ensure_directory_exists(savePath)

full_subject_path = os.path.join(savePath, subject_feature_path.strip('/'))
ensure_directory_exists(full_subject_path)

In [6]:
# This function computes the EDA
def eda_stats(y):
    Fs = fs_dict['EDA']
    y_norm = (y - y.mean()) / y.std()
    [r, p, t, l, d, e, obj] = cvxEDA.cvxEDA(y_norm, 1. / Fs)
    return [r, p, t, l, d, e, obj]

In [7]:
# The main class that creates attributes and access each survey to extract info
class SubjectData:
    def __init__(self, main_path, subject_number):
        self.name = f'S{subject_number}'
        self.subject_keys = ['signal', 'label', 'subject']
        self.signal_keys = ['chest', 'wrist']
        self.chest_keys = ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
        self.wrist_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        file_path = os.path.join(main_path, self.name, f'{self.name}.pkl')
        
        try:
            with open(file_path, 'rb') as file:
                self.data = pickle.load(file, encoding='latin1')
            self.labels = self.data.get('label', [])
        except FileNotFoundError:
            self.data = {}  
            print(f"Data file not found: {file_path}")
        except Exception as e:
            self.data = {}  
            print(f"An error occurred while loading the data: {str(e)}")

    def get_wrist_data(self):
        try:
            wrist_data = self.data['signal']['wrist'].copy()  
            wrist_data['Resp'] = self.data['signal']['chest']['Resp']  
            return wrist_data
        except KeyError as e:
            print(f"Key error accessing wrist/chest data: {str(e)}")
            return {}  

    def get_chest_data(self):
        return self.data.get('signal', {}).get('chest', {})  # Safe access with default empty dict

    def extract_features(self):
        results = {
                key: get_statistics(self.get_wrist_data()[key].flatten(), self.labels, key)
                for key in self.wrist_keys
            }
        return results

In [8]:
# Helper methods
# https://github.com/MITMediaLabAffectiveComputing/eda-explorer/blob/master/load_files.py
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scisig.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=5):
    # Filtering Helper functions
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = scisig.lfilter(b, a, data)
    return y

def get_slope(series):
    linreg = scipy.stats.linregress(np.arange(len(series)), series )
    slope = linreg[0]
    return slope

def get_window_stats(data, label=-1):
    mean_features = np.mean(data)
    std_features = np.std(data)
    min_features = np.amin(data)
    max_features = np.amax(data)

    features = {'mean': mean_features, 'std': std_features, 'min': min_features, 'max': max_features,
                'label': label}
    return features


def get_net_accel(data):
    return (data['ACC_x'] ** 2 + data['ACC_y'] ** 2 + data['ACC_z'] ** 2).apply(lambda x: np.sqrt(x))


def get_peak_freq(x):
    f, Pxx = scisig.periodogram(x, fs=8)
    psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
    peak_freq = psd_dict[max(psd_dict.keys())]
    return peak_freq

In [9]:
# Acc feature extraction
# https://github.com/MITMediaLabAffectiveComputing/eda-explorer/blob/master/AccelerometerFeatureExtractionScript.py
def filterSignalFIR(eda, cutoff=0.4, numtaps=64):
    f = cutoff / (fs_dict['ACC'] / 2.0)
    FIR_coeff = scisig.firwin(numtaps, f)

    return scisig.lfilter(FIR_coeff, 1, eda)

In [ ]:
def compute_features(e4_data_dict, labels, norm_type=None):
    # Dataframes for each sensor type
    eda_df = pd.DataFrame(e4_data_dict['EDA'], columns=['EDA'])
    bvp_df = pd.DataFrame(e4_data_dict['BVP'], columns=['BVP'])
    acc_df = pd.DataFrame(e4_data_dict['ACC'], columns=['ACC_x', 'ACC_y', 'ACC_z'])
    temp_df = pd.DataFrame(e4_data_dict['TEMP'], columns=['TEMP'])
    label_df = pd.DataFrame(labels, columns=['label'])
    resp_df = pd.DataFrame(e4_data_dict['Resp'], columns=['Resp'])

    # Filter EDA
    eda_df['EDA'] = butter_lowpass_filter(eda_df['EDA'], 1.0, fs_dict['EDA'], 6)

    # Filter ACM
    for _ in acc_df.columns:
        acc_df[_] = filterSignalFIR(acc_df.values)

    # Adding indices for combination due to differing sampling frequencies
    eda_df.index = [(1 / fs_dict['EDA']) * i for i in range(len(eda_df))]
    bvp_df.index = [(1 / fs_dict['BVP']) * i for i in range(len(bvp_df))]
    acc_df.index = [(1 / fs_dict['ACC']) * i for i in range(len(acc_df))]
    temp_df.index = [(1 / fs_dict['TEMP']) * i for i in range(len(temp_df))]
    label_df.index = [(1 / fs_dict['label']) * i for i in range(len(label_df))]
    resp_df.index = [(1 / fs_dict['Resp']) * i for i in range(len(resp_df))]

    # Change indices to datetime
    eda_df.index = pd.to_datetime(eda_df.index, unit='s')
    bvp_df.index = pd.to_datetime(bvp_df.index, unit='s')
    temp_df.index = pd.to_datetime(temp_df.index, unit='s')
    acc_df.index = pd.to_datetime(acc_df.index, unit='s')
    label_df.index = pd.to_datetime(label_df.index, unit='s')
    resp_df.index = pd.to_datetime(resp_df.index, unit='s')

    # New EDA features
    r, p, t, l, d, e, obj = eda_stats(eda_df['EDA'])
    eda_df['EDA_phasic'] = r
    eda_df['EDA_smna'] = p
    eda_df['EDA_tonic'] = t
        
    # Combined dataframe - not used yet
    df = eda_df.join(bvp_df, how='outer')
    df = df.join(temp_df, how='outer')
    df = df.join(acc_df, how='outer')
    df = df.join(resp_df, how='outer')
    df = df.join(label_df, how='outer')
    df['label'] = df['label'].fillna(method='bfill')
    df.reset_index(drop=True, inplace=True)

    if norm_type is 'std':
        # std norm
        df = (df - df.mean()) / df.std()
    elif norm_type is 'minmax':
        # minmax norm
        df = (df - df.min()) / (df.max() - df.min())

    # Groupby
    grouped = df.groupby('label')
    baseline = grouped.get_group(1)
    stress = grouped.get_group(2)
    amusement = grouped.get_group(3)
    return grouped, baseline, stress, amusement

In [11]:
# This function gets each features and labels
def get_samples(data, n_windows, label):
    global feat_names
    global WINDOW_IN_SECONDS

    samples = []
    window_len = fs_dict['label'] * WINDOW_IN_SECONDS

    for i in range(n_windows):

        w = data[window_len * i: window_len * (i + 1)]

        net_acc = get_net_accel(w[['ACC_x', 'ACC_y', 'ACC_z']])
        w = pd.concat([w, net_acc.rename('net_acc')], axis=1)

        wstats = get_window_stats(data=w, label=label)

        x = pd.DataFrame(wstats).drop('label', axis=0)
        y = wstats['label']

        if feat_names is None or len(feat_names) != len(x.columns):
            feat_names = [f'{str(row)}_{str(col)}' for row in x.index for col in x.columns]

        wdf = pd.DataFrame(x.values.flatten()).T
        wdf.columns = feat_names
        wdf['label'] = y

        wdf['BVP_peak_freq'] = get_peak_freq(w['BVP'].dropna())
        wdf['TEMP_slope'] = get_slope(w['TEMP'].dropna())

        samples.append(wdf)

    return pd.concat(samples) if samples else pd.DataFrame()

In [12]:
# This function gets the data from each survey
def make_patient_data(subject_id):
    global savePath
    global WINDOW_IN_SECONDS

    subject = SubjectData(main_path='data/WESAD', subject_number=subject_id)

    e4_data_dict = subject.get_wrist_data()

    # norm type
    norm_type = None

    grouped, baseline, stress, amusement = compute_features(e4_data_dict, subject.labels, norm_type)

    n_baseline_wdws = int(len(baseline) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    n_stress_wdws = int(len(stress) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    n_amusement_wdws = int(len(amusement) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    baseline_samples = get_samples(baseline, n_baseline_wdws, 1)
    stress_samples = get_samples(stress, n_stress_wdws, 2)
    amusement_samples = get_samples(amusement, n_amusement_wdws, 0)

    all_samples = pd.concat([baseline_samples, stress_samples, amusement_samples])
    all_samples = pd.concat([all_samples.drop('label', axis=1), pd.get_dummies(all_samples['label'])], axis=1)
    all_samples.to_csv(f'{savePath}{subject_feature_path}/S{subject_id}_feats_4.csv')

    subject = None

In [13]:
# This function combines paths to store the Sx_feats_4.csv
def combine_files(subjects):
    df_list = []
    for subject_id in subjects:
        file_path = os.path.join(savePath, subject_feature_path, f'S{subject_id}_feats_4.csv')
        df = pd.read_csv(file_path, index_col=0)
        df['subject'] = subject_id
        df_list.append(df)

    combined_df = pd.concat(df_list)
    combined_df = transform_labels_to_single_column(combined_df)
    save_combined_data(combined_df)

# Transforms multi-column labels into a single column
def transform_labels_to_single_column(df):
    label_cols = ['0', '1', '2']
    df['label'] = df[label_cols].astype(str).apply(lambda row: ''.join(row), axis=1).apply(lambda x: x.index('1'))
    df.drop(label_cols, axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

# Save the combined file as may14_feats4.csv
def save_combined_data(df):
    output_file = os.path.join(savePath, 'may14_feats4.csv')
    df.to_csv(output_file)
    print(f'Saved combined data to {output_file}')
    
    display_sample_counts(df['label'])

# This function displays sample counts by print out each label and corresponding counts
def display_sample_counts(labels):
    counts = labels.value_counts()
    print('Number of samples per class:')
    for label, number in counts.items():
        label_name = int_to_label.get(label, f'Label {label}')
        print(f'{label_name}: {number}')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Convert and extract survey 2-11 features
if __name__ == '__main__':

    subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

    for patient in subject_ids:
        print(f'Processing data for S{patient}...')
        make_patient_data(patient)

    combine_files(subject_ids)
    print('Processing complete.')

In [ ]:
# REFERENCES:
# [1] Schmidt, Philip & Reiss, Attila & Duerichen, Robert & Marberger, Claus & Van Laerhoven, Kristof. (2018). Introducing WESAD, a Multimodal Dataset for Wearable Stress and Affect Detection. 400-408. 10.1145/3242969.3242985.  https://dl.acm.org/citation.cfm?doid=3242969.3242985

# [2] A Greco, G Valenza, A Lanata, EP Scilingo, and L Citi "cvxEDA: a Convex Optimization Approach to Electrodermal Activity Processing", IEEE Transactions on Biomedical Engineering, 2015DOI: 10.1109/TBME.2015.2474131, https://github.com/lciti/cvxEDA

# [3] Schmidt, P., Reiss, A., Duerichen, R., Marberger, C., Van Laerhoven, K., & WJMatthew. (Year). WESAD: A publicly available dataset for wearable stress and affect detection. GitHub. https://github.com/WJMatthew/WESAD
